In [3]:
from pathlib import Path

import matplotlib.pyplot as plt
import moments.LD
import numpy as np

ImportError: cannot import name 'Timestamp' from 'timestamp' (unknown location)

In [ ]:
Ne = 10_000
mu = 2e-8
r = 1.25e-5
theta = 4 * Ne * mu
rho = 4 * Ne * r

In [ ]:
1 / (4 * Ne), theta, rho

In [ ]:
moments_result = moments.LD.Demographics1D.snm(
    rho=np.arange(60) * 4 * Ne * r,
    theta=theta,
    # rho=np.arange(11) * 4 * Ne * r, theta=theta
)
print(moments_result.names())
print(moments_result)
moments_ld_stats = np.vstack(moments_result[:-1])
moments_d2 = moments_ld_stats[:, 0]
moments_dz = moments_ld_stats[:, 1]
moments_pi2 = moments_ld_stats[:, 2]

In [ ]:
sim_results = Path(
    "~/simulation-outputs/ld-branch-validation-c-sims/output-1586920/result"
).expanduser()

In [ ]:
sim_data = np.dstack([np.load(path)["arr_0"] for path in sim_results.iterdir()]).mean(2)
d2 = sim_data[0]
dz = sim_data[1]
pi2 = sim_data[2]

In [ ]:
def relative_error(observed, expected):
    return np.abs(observed - expected) / expected

In [ ]:
results = list(
    zip(
        [moments_d2, moments_dz, moments_pi2],
        [d2, dz, pi2],
        ["$D^2$", "$D_z$", "$\pi_2$"],
    )
)


def plot_results(results):
    fig, axes = plt.subplots(2, 3, figsize=(10, 6))
    fig.suptitle("Comparison of two-locus statistics from Moments and tskit")
    num_x_ticks = 5
    for i, (moments, tskit, label) in enumerate(results):
        xticklabels = [
            f"${int(i)}N_er$"
            for i in np.arange(len(moments), step=len(moments) // num_x_ticks)
            * 4
            * Ne
            * r
        ]
        axes[0, i].plot(moments, ".-", label="moments")
        axes[0, i].plot(tskit, ".-", label="tskit")
        axes[1, i].plot(relative_error(tskit, moments), ".-", label="Relative Error")
        axes[0, i].set_title(label)
        for ax in axes[:, i]:
            ax.legend()
            ax.set_xticks(
                np.arange(len(moments), step=len(moments) // num_x_ticks), xticklabels
            )
    fig.tight_layout()

In [ ]:
plot_results(results)

Compare to the original [python prototype validation](https://github.com/lkirk/tskit-ld/blob/4546a5d/validation/notebooks/BranchStatisticalValidation.ipynb)